## **TRANSFER LEARNING WORKFLOW KERAS AND TENSORFLOW 2**
###### 1. Freeze weights and modify the pre-trained model's input parameters.
###### 2. Image set preparation. 
###### 3. Model training.
###### 4. Save The Model.

##### **I ] MODIFY THE PRE-TRAINED MODEL'S INPUT PARAMETERS**

In [1]:
import tensorflow as tf


In [2]:
# Load Mobilenet pre-trained model
# V2 | keras.applications.mobilenet_v2.MobileNetV2()
# V1 | keras.applications.mobilenet.MobileNet()

model_mobilenet_v2 = tf.keras.applications.mobilenet_v2.MobileNetV2()

14540800/14536120 [==============================] - 0s 0us/step


In [3]:
# Check VG166 layers

model_mobilenet_v2.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [4]:
model_mobilenet_v2.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [5]:
#Check the type of model format

type(model_mobilenet_v2)

tensorflow.python.keras.engine.training.Model

In [6]:
# Convert to Keras model 
# Remove the final layer of the Sequential model. We need a lower count input (current input argument is 1000) 
# Conversion with Sequential API doesn't work. Functional Model API is used instead. 

from tensorflow.keras.layers import Input, Dense, Activation
from tensorflow.keras.models import Model

x = model_mobilenet_v2.layers[-2].output           # This line removes the final layer of 1000 inputs
predictions = Dense(2, activation='softmax')(x)    # This line adds the output/fina layer of 2 nodes/inputs with softmax as activation method 
k_model_mobilenet_v2 = Model(inputs=model_mobilenet_v2.input, outputs=predictions) 

In [7]:
k_model_mobilenet_v2.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
______________________________________________________________________________________________

__________________________________________________________________________________________________
block_10_expand_relu (ReLU)     (None, 14, 14, 384)  0           block_10_expand_BN[0][0]         
__________________________________________________________________________________________________
block_10_depthwise (DepthwiseCo (None, 14, 14, 384)  3456        block_10_expand_relu[0][0]       
__________________________________________________________________________________________________
block_10_depthwise_BN (BatchNor (None, 14, 14, 384)  1536        block_10_depthwise[0][0]         
__________________________________________________________________________________________________
block_10_depthwise_relu (ReLU)  (None, 14, 14, 384)  0           block_10_depthwise_BN[0][0]      
__________________________________________________________________________________________________
block_10_project (Conv2D)       (None, 14, 14, 96)   36864       block_10_depthwise_relu[0][0]    
__________

In [8]:
# Freeze all weights. 

for layer in k_model_mobilenet_v2.layers [:-1]:
  layer.trainable = False
  

In [9]:
# Check if the dense layer with input 1 is added. 
k_model_mobilenet_v2.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
______________________________________________________________________________________________

In [10]:
# Compile the new model
from tensorflow.keras.optimizers import Adam
k_model_mobilenet_v2.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


##### **II ] PREPARE THE IMAGE SET**

In [11]:
train_path= '../input/imagescatsdogs1k/train'
valid_path= '../input/imagescatsdogs1k/valid'
test_path= '../input/imagescatsdogs1k/test'

In [12]:
# Specfiy shuffle=False as argument in test_batches for testing in confusion matrix

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(train_path, target_size=(224, 224), classes=['dogs', 'cats'], batch_size=100)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(valid_path, target_size=(224, 224), classes=['dogs', 'cats'], batch_size=20)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(test_path, target_size=(224, 224), classes=['dogs', 'cats'], batch_size=20)

Found 2178 images belonging to 2 classes.
Found 404 images belonging to 2 classes.
Found 407 images belonging to 2 classes.


##### III ] TRAIN AND SAVE THE MODEL

In [13]:
k_model_mobilenet_v2.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
k_model_mobilenet_v2.fit_generator(train_batches, steps_per_epoch=2, validation_data=valid_batches, validation_steps=2, epochs=20, verbose=2)

Train for 2 steps, validate for 2 steps
Epoch 1/20
2/2 - 15s - loss: 0.8832 - accuracy: 0.4150 - val_loss: 0.7225 - val_accuracy: 0.5750
Epoch 2/20
2/2 - 10s - loss: 0.8924 - accuracy: 0.4326 - val_loss: 0.7015 - val_accuracy: 0.6000
Epoch 3/20
2/2 - 10s - loss: 0.8682 - accuracy: 0.4500 - val_loss: 0.6810 - val_accuracy: 0.6250
Epoch 4/20
2/2 - 11s - loss: 0.8137 - accuracy: 0.4800 - val_loss: 0.6611 - val_accuracy: 0.6750
Epoch 5/20
2/2 - 11s - loss: 0.8049 - accuracy: 0.4950 - val_loss: 0.6425 - val_accuracy: 0.6750
Epoch 6/20
2/2 - 11s - loss: 0.7938 - accuracy: 0.4650 - val_loss: 0.6244 - val_accuracy: 0.6750
Epoch 7/20
2/2 - 11s - loss: 0.8104 - accuracy: 0.4750 - val_loss: 0.6071 - val_accuracy: 0.6750
Epoch 8/20
2/2 - 10s - loss: 0.7094 - accuracy: 0.5600 - val_loss: 0.5907 - val_accuracy: 0.7000
Epoch 9/20
2/2 - 15s - loss: 0.6762 - accuracy: 0.6250 - val_loss: 0.5752 - val_accuracy: 0.7250
Epoch 10/20
2/2 - 24s - loss: 0.7383 - accuracy: 0.5600 - val_loss: 0.5606 - val_accura

In [15]:
# k_model_mobilenet_v2.save('dogs_cats_xfer_mobilenet_v2.h5')

### **VERSIONS**
* v1.0 | Runs the basic
* v1.1 | Added preprocessing and modified steps and epoch